#**Results:**

In [1]:
import pandas as pd
results_ = {'Approach': ['XGBoost','Fastai','Catboost','LightGBM'],
        'RSME': [0.435097,0.399569,0.2522909698,0.4238094066114889]
        }
df = pd.DataFrame(results_, columns = ['Approach', 'RSME'])
df= df.sort_values(by='RSME', ascending=True)
df.head(8)

,Approach,RSME
2,Catboost,0.252291
1,Fastai,0.399569
3,LightGBM,0.423809
0,XGBoost,0.435097


#**XGBoost-Library:**

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Data Preperation

preprocessing data

In [4]:
rheindata = pd.read_csv("/content/drive/My Drive/fastai/data_rhein_incl_km(1).csv")
rheindata["DOC"].replace('', np.nan, inplace=True)
rheindata.dropna(subset=["DOC"], inplace=True)
rheindata = rheindata[["Kürzel","Date","year","month","day","Temp.(BAFU)","pH(BAFU)","Leitf. 25°C(BAFU)","Sauerstoff","Sauerstoffsätt.%","pH (EAWAG)","Leitf.20°C (EAWAG)","Alkalinität","Calcium","Magnesium","Nitrit","Nitrat","Ammonium","Kjeldahl-Stickstoff","Gesamt-Stickstoff","DRP","Gesamt-Phosphor","Chlorid","Kieselsäure","Sulfat","Natrium","Kalium","Eisen","Chem.Sauerstoffbedarf","TOC","DOC","Schwebstoffe","Chrom","Zink","Kupfer","Blei","Cadmium","Nickel","Quecksilber"]]
rheindata.head()

,Kürzel,Date,year,month,day,Temp.(BAFU),pH(BAFU),Leitf. 25°C(BAFU),Sauerstoff,Sauerstoffsätt.%,pH (EAWAG),Leitf.20°C (EAWAG),Alkalinität,Calcium,Magnesium,Nitrit,Nitrat,Ammonium,Kjeldahl-Stickstoff,Gesamt-Stickstoff,DRP,Gesamt-Phosphor,Chlorid,Kieselsäure,Sulfat,Natrium,Kalium,Eisen,Chem.Sauerstoffbedarf,TOC,DOC,Schwebstoffe,Chrom,Zink,Kupfer,Blei,Cadmium,Nickel,Quecksilber
0,RE,26.05.1975,1975,5,26,NaN,NaN,NaN,NaN,NaN,8.0,270.0,2.32,48.0960,6.3232,0.02,0.4,0.05,NaN,NaN,0.04,0.09,4.4,3.0,29.0,4.8,1.4,0.04,NaN,2.6,1.4,11.0,0.5,5.0,0.5,0.5,0.1,NaN,NaN
1,RE,02.06.1975,1975,6,2,13.7,NaN,NaN,NaN,NaN,8.2,280.0,2.32,50.5008,5.8368,0.03,0.7,0.03,NaN,NaN,0.07,0.18,5.6,3.0,30.0,4.7,1.3,0.09,NaN,2.2,1.8,6.5,0.5,10.0,2.0,1.0,0.1,NaN,NaN
2,RE,09.06.1975,1975,6,9,13.4,NaN,NaN,NaN,NaN,8.1,290.0,2.60,51.3024,5.8368,0.03,0.7,0.01,NaN,NaN,0.03,0.18,6.0,4.0,29.0,4.8,1.3,0.06,NaN,2.2,1.8,6.2,2.0,10.0,1.0,4.0,0.1,NaN,NaN
3,RE,16.06.1975,1975,6,16,14.9,NaN,NaN,NaN,NaN,8.2,280.0,2.44,48.8976,7.2960,0.02,0.7,0.06,NaN,NaN,0.04,0.18,5.4,4.0,29.0,4.9,1.3,0.07,NaN,2.4,1.8,1.2,0.5,10.0,1.0,1.0,0.1,NaN,NaN
4,RE,23.06.1975,1975,6,23,14.4,NaN,NaN,NaN,NaN,7.9,270.0,2.48,53.7072,5.8368,0.02,0.8,0.02,NaN,NaN,0.04,0.15,5.4,4.0,29.0,4.6,1.5,0.16,NaN,3.4,1.9,22.0,1.0,10.0,2.0,2.0,0.1,NaN,NaN


In [0]:
X = rheindata[["Temp.(BAFU)","pH(BAFU)","Leitf. 25°C(BAFU)",
                "Sauerstoff","Sauerstoffsätt.%","pH (EAWAG)",
                "Leitf.20°C (EAWAG)","Calcium",
                "Magnesium","Nitrit","Nitrat",
                "Kjeldahl-Stickstoff","Gesamt-Stickstoff",
                "Gesamt-Phosphor","Chlorid","Sulfat",
                "Kalium",
                "Schwebstoffe","Kupfer",
                "Quecksilber"]]
y = rheindata[["DOC"]]

defining train and test split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [0]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [15]:
data_dmatrix = xgb.DMatrix(data=X,label=y)
data_dmatrix

# Model

varying different kind of parameters (f.ex. iterations, learing rate and depth) to find a minimal RSME

In [0]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.6, learning_rate = 0.4,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [11]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

[14:27:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [13]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.474842


In [19]:
params = {"objective":"reg:linear",'colsample_bytree': 0.6,'learning_rate': 0.01,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=1000,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
cv_results

[14:33:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:33:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:33:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,1.256150,0.015654,1.255989,0.032041
1,1.245639,0.015546,1.245432,0.032366
2,1.235197,0.015537,1.235141,0.032521
3,1.224880,0.015564,1.224953,0.032612
4,1.214766,0.015589,1.214927,0.032683
...,...,...,...,...
995,0.355731,0.006996,0.429542,0.027128
996,0.355690,0.007006,0.429525,0.027118
997,0.355648,0.006998,0.429524,0.027111
998,0.355610,0.006993,0.429521,0.027102


In [21]:
params = {"objective":"reg:linear",'colsample_bytree': 0.6,'learning_rate': 0.05,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=1000,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
cv_results

[14:39:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,1.213755,0.015434,1.214197,0.032796
1,1.163700,0.014940,1.163958,0.034373
2,1.116189,0.014850,1.117207,0.035168
3,1.071458,0.015022,1.072978,0.035475
4,1.029659,0.015119,1.031746,0.035760
...,...,...,...,...
271,0.344752,0.006128,0.430139,0.026001
272,0.344642,0.006186,0.430122,0.026013
273,0.344539,0.006159,0.430118,0.026019
274,0.344446,0.006121,0.430117,0.026049


In [22]:
params = {"objective":"reg:linear",'colsample_bytree': 0.6,'learning_rate': 0.005,
                'max_depth': 10, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=1000,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
cv_results

[14:39:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,1.261465,0.015681,1.261235,0.031943
1,1.256177,0.015628,1.255930,0.032110
2,1.250894,0.015621,1.250728,0.032184
3,1.245646,0.015633,1.245548,0.032228
4,1.240466,0.015645,1.240430,0.032265
...,...,...,...,...
995,0.365617,0.005698,0.438992,0.029315
996,0.365545,0.005714,0.438983,0.029309
997,0.365483,0.005712,0.438963,0.029304
998,0.365408,0.005735,0.438945,0.029298
